<a href="https://www.kaggle.com/code/houmannorasteh/fork-of-t5-fine-tuning?scriptVersionId=164693993" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install transformers
! pip install torch

### importing libraries 

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from itertools import zip_longest

### defining general variables

In [3]:
#defining general valriables throughout the whole notebook
EPOCH = 10
batch_size = 16
max_input_length = 64
max_label_length = 8
MODEL_LINK = "google/flan-t5-small"

In [4]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_LINK, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(MODEL_LINK).to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### reading dataset and cleaning it

In [12]:
# a funciton to read data off of a database link

def get_data(address):
    lines = []
    with open(address) as file:
        for line in file:
            x = json.loads(line)
            lines.append(x)
    sentences, orl, sep_sentences = [], [], []
    for i in range(len(lines)):
        sep_sentences.append(lines[i]['sentences'])
        sentences.append(' '.join(lines[i]['sentences']))
        orl.append(lines[i]['orl'])
    dataframe = pd.DataFrame({'sentence': sentences, 'orl': orl, 'sep_sent': sep_sentences})
    return dataframe

### getting training data into df and dividing each of {agent, target, dse} elements

In [37]:
df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.train0.conll.json")
df

,sentence,orl,sep_sent
0,The Kimberley Provincial Hospital said it woul...,"[[6, 8, 0, 3, AGENT], [6, 8, 6, 8, DSE], [6, 8...","[The, Kimberley, Provincial, Hospital, said, i..."
1,Saeed said indications were that those tests w...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...","[Saeed, said, indications, were, that, those, ..."
2,He said it was his opinion that the patient --...,"[[4, 5, 0, 0, AGENT], [4, 5, 4, 5, DSE], [4, 5...","[He, said, it, was, his, opinion, that, the, p..."
3,The woman was admitted to the hospital on Satu...,"[[10, 10, 0, 1, AGENT], [10, 10, 10, 10, DSE],...","[The, woman, was, admitted, to, the, hospital,..."
4,`` Since our technical equipment is far from p...,"[[22, 22, 2, 4, TARGET], [22, 22, 10, 10, TARG...","[``, Since, our, technical, equipment, is, far..."
...,...,...,...
2444,Benjamin Franklin Federal Savings & Loan Assoc...,"[[9, 9, 8, 8, AGENT], [9, 9, 9, 9, DSE]]","[Benjamin, Franklin, Federal, Savings, &, Loan..."
2445,thrift said the restructuring should help it m...,"[[1, 1, 1, 1, DSE], [1, 1, 2, 3, TARGET]]","[thrift, said, the, restructuring, should, hel..."
2446,Details of the restructuring wo n't be made fi...,"[[11, 11, 10, 10, AGENT], [11, 11, 11, 11, DSE]]","[Details, of, the, restructuring, wo, n't, be,..."
2447,"Jay Stevens , an analyst with Dean Witter Reyn...","[[12, 12, 11, 11, AGENT], [12, 12, 12, 12, DSE]]","[Jay, Stevens, ,, an, analyst, with, Dean, Wit..."


In [39]:
# Function to organize the data into separate columns
def list_of(atributs, requested_atr):
    requested_list = []
    for sublist in atributs:
        if sublist[-1] == requested_atr:
            requested_list.append(sublist)
    return requested_list

def organize_data(atributs, sentence):
    AGENT, DSE, TARGET = '', '', ''
    target_flag, agent_flag = False, False
    for sublist in atributs:
        if sublist[-1] == 'DSE':
            dse_start = int(sublist[0])
            dse_end = int(sublist[1] + 1)
            DSE += ' '.join(sentence[dse_start:dse_end]) + '|'
            
            target_flag = False
            # looking for the targets and agents of this dse
            for sub_sublist in list_of(atributs, 'TARGET'):
                if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                    target_start = int(sub_sublist[2])
                    target_end = int(sub_sublist[3] + 1)
                    TARGET += ' '.join(sentence[target_start:target_end]) + '|'
                    target_flag = True
            if not target_flag:
                TARGET += ' |'
            
            agent_flag = False
            for sub_sublist in list_of(atributs, 'AGENT'):
                if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                    agent_start = int(sub_sublist[2])
                    agent_end = int(sub_sublist[3] + 1)
                    AGENT += ' '.join(sentence[agent_start:agent_end]) + '|'
                    agent_flag = True
            if not agent_flag:
                AGENT += ' |'
    return AGENT, DSE, TARGET

# Organize tarin data into diffrent columns
for i in range(len(df)):
    agent, dse, target = organize_data(df['orl'][i], df['sep_sent'][i])
    df.loc[i, 'agent'] = agent
    df.loc[i, 'dse'] = dse
    df.loc[i, 'target'] = target

In [40]:
print(df.iloc[2435]['sentence'])
print(df.iloc[2435]['orl'])
print(f"target:{df.iloc[2435]['target']}")
print(f"dse: {df.iloc[2435]['dse']}")
print(f"agent: {df.iloc[2435]['agent']}")

The transaction has been approved by Kyle 's board , but requires the approval of the company 's shareholders .
[[2, 4, 0, 1, 'TARGET'], [2, 4, 2, 4, 'DSE'], [2, 4, 6, 8, 'AGENT'], [13, 13, 13, 13, 'DSE'], [13, 13, 15, 18, 'AGENT']]
target:The transaction| |
dse: has been approved|approval|
agent: Kyle 's board|the company 's shareholders|


In [ ]:
# for i in range(len(df)):
#     if (int(len(df.iloc[i]['orl'])) >= 6):
#         print(i)


print(df.iloc[2430]['sentence'])
print(df.iloc[2430]['orl'])
print(df.iloc[2430])

In [ ]:
df.drop(columns='sep_sent')

In [ ]:
def pipeDivider(pipedString):
    listOfItems = []
    listOfItems = pipedString.split('|')[:-1]
    return listOfItems

In [ ]:
# it is here to iterate over every single dse -inner loop- of every sentence of the database -outer loop-
# (very inefficient, i know. i havn't figured out what to do yet)
train_df = pd.DataFrame(columns=['target_prompt', 'agent_prompt', 'target', 'agent'])

for i in range(len(df)):
    dse_list = []
    dse_list = pipeDivider(str(df.iloc[i]['dse']))
    target_list = pipeDivider(str(df.iloc[i]['target']))
    agent_list = pipeDivider(str(df.iloc[i]['agent']))
    for j in range(len(dse_list)):
        last_row = int(len(train_df))+1
        train_df.loc[last_row, 'target_prompt'] = f"Sentence is: {df.iloc[i]['sentence']} Find target for this dse: {dse_list[j]}"
        train_df.loc[last_row, 'agent_prompt'] = f"Sentence is: {df.iloc[i]['sentence']} Find agent for this dse: {dse_list[j]}"
        train_df.loc[last_row, 'target'] = target_list[j]
        train_df.loc[last_row, 'agent'] = agent_list[j]

train_df.head(-3)

In [ ]:
print(train_df.iloc[3]['target_prompt'])
print(train_df.iloc[4]['target_prompt'])
print(train_df.iloc[5]['target_prompt'])
print(train_df.iloc[6]['target_prompt'])

In [ ]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length4text, max_length4label):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length4text = max_length4text
        self.max_length4label = max_length4label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['sentence'][idx]
        agent = self.data['agent'][idx]
        dse = self.data['dse'][idx]
        target = self.data['target'][idx]
        text_encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        #agent
        agent_encoding = self.tokenizer(agent, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        #dse
        dse_encoding = self.tokenizer(dse, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        #target
        target_encoding = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        return {
            #text
            'input_ids': text_encoding['input_ids'].squeeze(),
            'attention_mask': text_encoding['attention_mask'].squeeze(),
            #agent
            'agent_id': agent_encoding['input_ids'].squeeze(),
            'agent_mask': agent_encoding['attention_mask'].squeeze(),
            #dse
            'dse_id': dse_encoding['input_ids'].squeeze(),
            'dse_mask': dse_encoding['attention_mask'].squeeze(),
            #target
            'target_id': target_encoding['input_ids'].squeeze(),
            'target_mask': target_encoding['attention_mask'].squeeze()
        }

In [ ]:
# Create a data loader
train_dataset = CustomDataset(df[:2500], tokenizer, max_length4text=max_input_length, max_length4label=max_label_length)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CustomDataset(dev_df, tokenizer, max_length4text=max_input_length, max_length4label=max_label_length)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def cleaner(str_item):
    final_string_list = []
    string_list = str_item.split('|')
    for i in string_list:
        j = i.replace(' ', '')
        if len(j) > 0:
            # data cleaning
            final_string_list.append(j)
            
    return final_string_list


In [ ]:
cleaner("a|b|  |")

In [ ]:
def f1_calculator(pred_list, actual_list):
    cleaned_pred, cleaned_actual = [], []
    matched = 0
    for i in range(len(pred_list)):
        pred_items = cleaner(pred_list[i])
        actual_items = cleaner(actual_list[i])
        # function:
        for j in range(len(actual_items)):
            print(actual_items[j])
            if actual_items[j] in pred_items:
                matched += 1
                print(f'** actual ** \n')
        cleaned_pred.extend(pred_items)
        cleaned_actual.extend(actual_items)
    
    prediction_len = len(cleaned_pred)
    actual_len = len(cleaned_actual)
    print(f'matched: {matched}, prediction_len:{prediction_len}, actual_len:{actual_len} \n')
    try:
        precision = (matched / prediction_len)
        recall = (matched / actual_len)
        f1 = (2 * (precision * recall)) / (precision + recall)
    except:
        f1 = 0
    return f1

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        true_counted, prediction_counted, actual_counted = 0,0,0
        matched, prediction_len, actual_len = 0,0,0
        actual_list, prediction_list, f1 = [], [], []
        for batch_idx, batch in enumerate(dataloader):
            # Move data to the specified device
            # batch = {key: value.to('cuda') for key, value in batch.items()}

            # Forward pass
            ids = batch['input_ids']
            mask = batch['attention_mask']
            target_id = batch['dse_id']
            
            actuals = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in target_id]
            
            output = generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=64, 
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in output]
            
            actual_list.extend(actuals)
            prediction_list.extend(preds)
    return f1_calculator(prediction_list, actual_list)

In [ ]:
model.train()
for epoch in range(EPOCH):
    losses = []
    print(f'epoch: {epoch} \n')
    for batch in train_data_loader:
        inputs = batch['input_ids']
        attention_mask = batch['attention_mask']
        dse_id = batch['dse_id']
        dse_mask = batch['dse_mask']

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask, labels=dse_id)
        loss = outputs.loss
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    f1 = evaluate_model(model, val_data_loader)
    print(f'loss: {np.mean(losses)}, f1 validation:{f1} \n end of epoch{epoch}. \n')

In [ ]:
for epoch in range(EPOCH):
    print(f'epoch: {epoch} \n')
    accuracy = evaluate_model(model, val_data_loader)
    print(f'accuracy: {accuracy}')

In [ ]:
input_text = "find expression of the sentence: The Palestinians want nothing from Washington but to understand their cause and stand beside right and justice ."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))